# Secret Sharing

Secret sharing is a technique widely used in secure multiparty computation. It relies on sharing the responsability of a certain information by simply splitting it in several shares. As always we explain by example.

In the first part we will work on additive secret sharing



# Additive Secret Sharing

As said previously, secret sharing consists (at least at this stage) in splitting the information into shares. First let's choose a random integer all the parties agree on:

In [1]:
from crypt import RandomPrime
from random import seed, randrange

seed(1)

bits = 64
p = RandomPrime(bits, 100)
print(f"We've chosen a random prime p={p}")

We've chosen a random prime p=13931278444523239403


We choose the number of parties $N$ that will participate in the calculation and the secret we want to split. Then we generate N-1 random numbers in between 0 and $p$ and finally substract the secret to generate the Nth share, this way when we sum the shares they will add up to the secret.

In [2]:
N = 5
secret = 100

def Share(secret, N, p):
    # generate N-1 random shares
    shares = [randrange(p) for _ in range(N-1)]
    shares.append((secret-sum(shares))%p)
    return shares

def ReconstructShares(shares, p):
    return (sum(shares)%p)

shares = Share(secret, N, p)
reconstruction = ReconstructShares(shares, p)
print(f"Splitting to shares:\n\nsecret={secret}\nsplits={shares}\nsum_shares={reconstruction}")

Splitting to shares:

secret=100
splits=[2988738511823670261, 8317795916909459611, 12560204332531295242, 2932013314605955158, 1063804813176098634]
sum_shares=100


What we've done here is splitting the secret into N users so that when they all share their shares they can reconstruct the secret, no $N-1$ parties can reconstruct!. Beautiful right?

Now, how much information can $N-1$ parties gain from the original secret? Here's the best, there's no information. This is true due to the randomicity of this process, since $N-1$ are chosen completely at random from a uniform distribution they contain no information and by construction the Nth number also. This concept is exactly the same as the one time pad encoding we have seen previously. 

It is needed all the shares to reconstruct the value, what if there is one malitious? We cannot reconstruct back the secret. This is something we will explore in Shamir Secret Sharing.

## Defining sum on shares

We can define addition and substraction on two vectors of shares. 

In [3]:
def SharesAdd(x, y, p):
    return [(i+j)%p for i, j in zip(x,y)]

def SharesSubstract(x, y, p):
    return [(i-j)%p for i, j in zip(x,y)]

It is possible to outsource a summation or a substraction by sharing the values of two integers in the following way. Supplose that Alice has the value $x$ and Bob the value $y$, both want to calculate $x+y$ without the other knowing their value (actually, if they know $x+y$ and one value they can know the other value). First what they do is split into $N$ shares:

In [4]:
x = randrange(0, p//2-1)
y = randrange(0, p//2-1)

print(f"I want to add secretly x and y:\nx={x}\ny={y}\n")
print(f"First task is to generate random shares for x and y and send them to the {N} outsourced parties:")
x_shares = Share(x, N, p)
y_shares = Share(y, N, p)

x_rec = ReconstructShares(x_shares, p)
y_rec = ReconstructShares(y_shares, p)

print(f"x_shares = {x_shares} which reconstructed is {x_rec}")
print(f"y_shares = {y_shares} which reconstructed is {y_rec}\n")

I want to add secretly x and y:
x=4013115311796737717
y=5007603362139683160

First task is to generate random shares for x and y and send them to the 5 outsourced parties:
x_shares = [10149566149063680291, 13126905008247241882, 5800758372187592044, 3829813701094019164, 12899907414773922545] which reconstructed is 4013115311796737717
y_shares = [5855298019442606879, 502905284296215807, 13401259479117932483, 5907593270417480351, 7203104197911926446] which reconstructed is 5007603362139683160



Now Alice has to send each share of $x$ and Bob its shares of $y$ to a different party

In [5]:
for i, (xs, ys) in enumerate(zip(x_shares, y_shares)):
    print(f"Party {i} gets the values x_{i}={xs} and y_{i}={ys}")

Party 0 gets the values x_0=10149566149063680291 and y_0=5855298019442606879
Party 1 gets the values x_1=13126905008247241882 and y_1=502905284296215807
Party 2 gets the values x_2=5800758372187592044 and y_2=13401259479117932483
Party 3 gets the values x_3=3829813701094019164 and y_3=5907593270417480351
Party 4 gets the values x_4=12899907414773922545 and y_4=7203104197911926446


Each party calculates their sum of their shares and send it back to Alice

In [6]:
x_y_sum_shares = SharesAdd(x_shares, y_shares, p)
print(f"The final sum of shares is: \nx_y_sum={x_y_sum_shares}")

The final sum of shares is: 
x_y_sum=[2073585723983047767, 13629810292543457689, 5270739406782285124, 9737406971511499515, 6171733168162609588]


Now is time to reconstruct the shares and find the secret

In [7]:
x_y_sum = ReconstructShares(x_y_sum_shares, p)

assert(x+y==x_y_sum)

print(f"The reconstruction of the secret is {x_y_sum}, the sum of x={x} and y={y}")

The reconstruction of the secret is 9020718673936420877, the sum of x=4013115311796737717 and y=5007603362139683160


Done!. We've split two secrets, sent them to different parties that sum their shares and finally we get each contribution in a vector. We just need to add this vector to reconstruct as we did.

### Problems with overflow

You may notice that we've generated two random numbers in the range of $0$ to $p//2-1$, we do this to avoid overflow, this is, if we add two numbers and the sum is larger than $p$ then when we apply the modulo operation we are back to square one and the sum has no meaning. Let me show an example

In [8]:
from crypt import isPrime

# Choose a small prime
q = 17 
assert isPrime(q), f"{q} is not prime, please choose a small prime"
print(f"Prime number selected: q={q}")

# Choose two numbers smaller than p so that when are summed are larger than q
x, y = 10, 8

print(f"Chosen x, y = {x}, {y}")
print(f"Modulo sum of x, y = (x+y)mod q = {(x+y)%q}")

if x+y!=(x+y)%q:
    print("Overflow!")

Prime number selected: q=17
Chosen x, y = 10, 8
Modulo sum of x, y = (x+y)mod q = 1
Overflow!


Let's do the same sum but using the shares

In [9]:
x_shares = Share(x, N, q)
y_shares = Share(y, N, q)

x_rec = ReconstructShares(x_shares, q)
y_rec = ReconstructShares(y_shares, q)

print(f"x_shares = {x_shares} which reconstructed is {x_rec}")
print(f"y_shares = {y_shares} which reconstructed is {y_rec}\n")

x_shares = [14, 12, 10, 12, 13] which reconstructed is 10
y_shares = [2, 2, 10, 14, 14] which reconstructed is 8



In [10]:
for i, (xs, ys) in enumerate(zip(x_shares, y_shares)):
    print(f"Party {i} gets the values x_{i}={xs} and y_{i}={ys}")

Party 0 gets the values x_0=14 and y_0=2
Party 1 gets the values x_1=12 and y_1=2
Party 2 gets the values x_2=10 and y_2=10
Party 3 gets the values x_3=12 and y_3=14
Party 4 gets the values x_4=13 and y_4=14


In [11]:
x_y_sum_shares = SharesAdd(x_shares, y_shares, q)
x_y_sum = ReconstructShares(x_y_sum_shares, q)

print(f"The reconstruction of the secret is {x_y_sum}, the sum of x={x} and y={y}")
if x+1!=x_y_sum:
    print(f"Overflow!, i.e. bad reconstruction.")

The reconstruction of the secret is 1, the sum of x=10 and y=8
Overflow!, i.e. bad reconstruction.


## Defining multiplication on shares

It is possible to define a multiplication in this additive scheme, we will use N=3 parties for simplicity.

The initial step is the same as before, Alice splits $x$ into shares while Bob does the same with $y$. Then they both send the values to each party, in this case:

Party 0: $x_1$, $x_2$, $y_1$, $y_2$

Party 1: $x_2$, $x_0$, $y_2$, $y_0$

Party 2: $x_0$, $x_1$, $y_0$, $y_1$

Nottice that, no party can fully reconstruct x or y, however here if two parties agree they can reconstruct a variable. For instance, if Party 0 and Party 1 are both malitious they can fully reconstruct x and y.


Recall that x = [$x_0$, $x_1$, $x_2$] and y = [$y_0$, $y_1$, $y_2$]
and 

$$x * y = x_1*y_1+x_1*y_2+x_2*y_1+$$
$$x_2*y_0+x_2*y_2+x_0*y_2+$$
$$+x_0*y_0+x_0*y_1+x_1*y_0$$

**WARNING**: This is not well rendered on GitHub. Please download the repo and do it in your local machine!!

Party 0 can compute line 0 (containing indexes 1 and 2), Party 1 can compute line 1 (contain indexes 0 and 2) and party 2 calculates line 2 (again contains indexes 0 and 1). Recall the permutation nature of this calculation. We define the following quantities:


$$z_0 = x_1*y_1+x_1*y_2+x_2*y_1$$
$$z_1 = x_2*y_0+x_2*y_2+x_0*y_2$$
$$z_2 = x_0*y_0+x_0*y_1+x_1*y_0$$

whose sum is $x*y$

$$x*y = z = z_0 + z_1 + z_2$$

In [12]:
x = 13
y = 17

# Assuming 3 parties all the time
# Definition of the crossed terms
def indexes(i):
    # crossed intexes for 3 parties
    if i==0:
        return [1, 2]
    if i==1:
        return [2, 0]
    if i==2:
        return [0, 1]
    return None

def SharesToi(v, party):
    #give the shares of v to party (party 0, 1 or 2)
    i, j = indexes(party)
    return [v[i], v[j]]

def LocalMult(shares, p):   
    #local multiplication and reshare
    return Share(shares[0]*shares[2]+shares[0]*shares[3]+shares[1]*shares[2], 3, p)

From the above code the indexes function gives the indexes of $x$ and $y$ to be shared with party i 

In [13]:
for k in range(3):
    i, j = indexes(k)
    print(f"Shares of Party {k}: x{i},x{j},y{i},y{j}")

Shares of Party 0: x1,x2,y1,y2
Shares of Party 1: x2,x0,y2,y0
Shares of Party 2: x0,x1,y0,y1


Now, Alice (Party 0) and Bob (Party 1) split their values of $x$ and $y$ and send their needed shares between themselves.

Alice (Party 0) sends to Bob (Party 1): $x_2$ and $x_0$

Alice (Party 0) sends to Party 3: $x_0$ and $x_1$

Bob (Party 1) sends to Alice (Party 0): $y_1$ and $y_2$

Bob (Party 1) sends to Party 3: $y_0$ and $y_1$

This is done in the following chunk of code (recall that any of the participants cannot reconstruct either $x$ or $y$).

In [14]:
x_shares = Share(x, 3, p)
y_shares = Share(y, 3, p)

party_shares_send = []

for party in range(3):
    sharex0, sharex1 = SharesToi(x_shares, party)
    sharey0, sharey1 = SharesToi(y_shares, party)
    
    party_shares_send.append([sharex0, sharex1, sharey0, sharey1])
    print(f"Party {party} gets {party_shares_send[-1]}")

Party 0 gets [8650049126189053188, 668384414172824918, 3379863109552534974, 3987565969275602706]
Party 1 gets [668384414172824918, 4612844904161361310, 3987565969275602706, 6563849365695101740]
Party 2 gets [4612844904161361310, 8650049126189053188, 6563849365695101740, 3379863109552534974]


Now each party performs the computation on their shares, meaning:

$$z_0 = x_1*y_1+x_1*y_2+x_2*y_1$$
$$z_1 = x_2*y_0+x_2*y_2+x_0*y_2$$
$$z_2 = x_0*y_0+x_0*y_1+x_1*y_0$$

This is done by the function LocalMult defined above. Recall that LocalMult do not return make public directly $z_i$ but instead it secretly shares the output into 3. Then all these shares are shared among the parties. Look at the following example

In [15]:
z = []
for i in range(3):
    z.append(LocalMult(party_shares_send[i], p))
    print(f"Party {i} does LocalMult and splits the result: {z[-1]}")


print("\nParties start a phase of communication of the above shares: ")
z_reshared = []
for i in range(3):
    z_reshared.append([z[0][i], z[1][i], z[2][i]])
    print(f"Party {i} gets {z_reshared[-1]}")
    
print("\nEach party sums their shares and send it to the others")
final_value = []
for i in range(3):
    s = sum(z_reshared[i])%p
    print(f"Party {i} sums to {s}")
    final_value.append(s)


z = ReconstructShares([final_value[0]+final_value[1]+final_value[2]], p)

assert(z==x*y)


print(f"\n\nThe sum of {final_value[0], final_value[1], final_value[2]} \
modulo {p} is {z}, that is the multiplication of {x} and {y} as expected")

Party 0 does LocalMult and splits the result: [3833914232821674549, 3674884782689130710, 3344564429637056239]
Party 1 does LocalMult and splits the result: [6649379561686619790, 1649412028649022347, 10168560706762371730]
Party 2 does LocalMult and splits the result: [13895686642489791361, 1669217059871504548, 10839494333485786559]

Parties start a phase of communication of the above shares: 
Party 0 gets [3833914232821674549, 6649379561686619790, 13895686642489791361]
Party 1 gets [3674884782689130710, 1649412028649022347, 1669217059871504548]
Party 2 gets [3344564429637056239, 10168560706762371730, 10839494333485786559]

Each party sums their shares and send it to the others
Party 0 sums to 10447701992474846297
Party 1 sums to 6993513871209657605
Party 2 sums to 10421341025361975125


The sum of (10447701992474846297, 6993513871209657605, 10421341025361975125) modulo 13931278444523239403 is 221, that is the multiplication of 13 and 17 as expected


We've done it!. We applied a protocol to do secure multiplication on additive secret sharing with three parties. Now you may ask... Can't we do multiplication with just the original data holders Alice and Bob? (just 2 parties) The answer is no, we need the third party to compute other cross terms and keep $x$ and $y$ secret in the information-theoretic sense. However, there are other aproaches based on cryptography (i.e. the adversary can get the information with enough computing power)

## Dating app

An example where we could use multiplication in additive sharing is a dating app. Suppose that Alice and Bob meet online where they can see their pictures and description. Both can decide whether they like the other person (1) or not (0). They have a match if they both like each other, see the following table:

| Alice | Bob | Match |
|:-----:|:---:|:-----:|
|   0   |  0  |   0   |
|   1   |  0  |   0   |
|   0   |  1  |   0   |
|   1   |  1  |   1   |

Imagine that Bob likes Alice but he doesn't want Alice to know unless she likes him too. If Bob likes her (1) then when the secret is revealed for both if she has chosen 0 then she will never know whether Bob liked her. That way Bob is confident to set 1 to her.

Let's to the example

In [16]:
x = 0 #Alice's does not like Bob
y = 1 #Bob likes Alice

x_shares = Share(x, 3, p)
y_shares = Share(y, 3, p)

party_shares_send = []

for party in range(3):
    sharex0, sharex1 = SharesToi(x_shares, party)
    sharey0, sharey1 = SharesToi(y_shares, party)
    
    party_shares_send.append([sharex0, sharex1, sharey0, sharey1])
    print(f"Party {party} gets {party_shares_send[-1]}")

Party 0 gets [6251302400758002017, 11016252108278469876, 757280478805453413, 8978724445824733995]
Party 1 gets [11016252108278469876, 10595002380010006913, 8978724445824733995, 4195273519893051996]
Party 2 gets [10595002380010006913, 6251302400758002017, 4195273519893051996, 757280478805453413]


In [17]:
z = []
for i in range(3):
    z.append(LocalMult(party_shares_send[i], p))


z_reshared = []
for i in range(3):
    z_reshared.append([z[0][i], z[1][i], z[2][i]])
    
final_value = []
for i in range(3):
    s = sum(z_reshared[i])%p
    final_value.append(s)


z = ReconstructShares([final_value[0]+final_value[1]+final_value[2]], p)

assert(z==x*y)


print(f"The sum of {final_value[0], final_value[1], final_value[2]} \
modulo {p} is {z}, that is the multiplication of {x} and {y} as expected")

The sum of (5328095093268161759, 309574326156819252, 8293609025098258392) modulo 13931278444523239403 is 0, that is the multiplication of 0 and 1 as expected


They both know that there is no match... Bob knows that Alice does not like him but Alice does not know what has Bob responded. If Bob meets Alice in person he will be relaxed that she knows nothing about his response.